In [1]:
# src: https://www.geeksforgeeks.org/fernet-symmetric-encryption-using-cryptography-module-in-python/
from cryptography.fernet import Fernet as fr
import pandas as pd
from getpass import getpass as gp

In [2]:
with open('securekey.txt','rb') as secure:
    keys = [i.strip() for i in secure]
    pwd_key = keys[0]
    usr_key = keys[1]
print(pwd_key,usr_key)

b'lVqK1Z97YksdG03JY8o0y3z9kKygyH8vob-QN0wxKkw=' b'Wj-8hNy59EyapiBqKLQqVKe3JsbFYnY9jqas8e-q8F8='


In [3]:
def S_encoder(var, encoder, txtfile): # self written encoder function.
# variable can be username or password
# encoder will be either spec_u_enc or spec_p_enc
# txtfile will be either user.txt to store username or pwd,txt to store pwd
    enc = encoder.encrypt(var.encode())
    with open(txtfile, 'ab') as txt:
        txt.write(enc)
        txt.write('\n'.encode())

def S_decode(spec_enc, txtfile):
    with open(txtfile,'rb') as txt:
        li = [spec_enc.decrypt(i).decode() for i in txt]
        return li

In [22]:
def update(user_files):
    
    if type(user_files) == str:
        csvFile = user_files
    else:
        csvFile = input(f'while of the following file {user_files} would u like to update? ')    
        while csvFile not in user_files:    
            csvFile = input(f'wrong input, select file from {user_files} ')
    import pandas as pd
    head = list(pd.read_csv(csvFile).columns)
    with open(csvFile,'a') as csv:
        row = 1
        control = input(f'would you like to update {row} row of data into budget? y/n for yes no')
        while control.lower()[0] =='y':
            data = [input(f'data for column {i} ?') for i in head]
            csv.write(','.join(data))
            csv.write('\n')
            row += 1
            control = input(f'would you like to continue updating {row} row of data into budget? y/n for yes no')

def read(user_files):
    import pandas as pd
    
    csvFile = input(f'while of the following file {user_files} would u like to read? ')
    while csvFile not in user_files:
        csvFile = input(f'wrong input, select file from {user_files} ')
    df = pd.read_csv(csvFile)
    return df

def create_empty_file(csvFile):
    head = [input(f'what is the heading for column {i}') for i in range(int(input('number of column headings required ')))]
    with open(csvFile , 'w') as csv:
        csv.write(','.join(head))
        csv.write('\n')
    print(f'{csvFile} created')

In [20]:
spec_p_enc = fr(pwd_key) # specialised encoder using pwd_key for pwd
spec_u_enc = fr(usr_key)# specialised encoder using usr_key for usr
checker = input('enter 0 for sign up and 1 for sign in ')
li = [str(0),str(1)]
while checker not in li:
    print(f'no action taken as action input of {checker} is invalid')
    checker = input('enter 0 for sign up and 1 for sign in ')
checker = int(checker)
if checker == 0:
    # sign up function
    user = input('your username: ')
    password = gp(prompt='your password')
    repwd = gp(prompt='re enter your password')
    while password != repwd:
        print('\nyour passwords do not match\n')
        password = gp(prompt='your password')
        repwd = gp(prompt='re enter your password')
    try:
        reg = ''
        li = S_decode(spec_u_enc, 'user.txt')
        while reg != True:
            if user not in li:
                S_encoder(user, spec_u_enc, 'user.txt')
                S_encoder(password, spec_p_enc, 'pwd.txt')
                reg = True
            else:
                print('this username is taken')
                user = input('a new username: ')
    except:
        S_encoder(user, spec_u_enc, 'user.txt')
        S_encoder(password, spec_p_enc, 'pwd.txt')
    proceed = 0
    
#####################################################################################################################
elif checker == 1:
    user = input('your username: ')
    password = gp(prompt='your password')
    li = S_decode(spec_u_enc, 'user.txt')
    while user not in li:
        print('wrong username ')
        user = input('your username: ')
    pi = S_decode(spec_p_enc, 'pwd.txt')
    while password != pi[li.index(user)]:
        print('wrong password')
        password = gp(prompt='your password')
    print('welcome back')
    proceed = 1
###########################################################################################################    
else:
    print('your input neither 0 nor 1 ')
    proceed = 0

enter 0 for sign up and 1 for sign in 1
your username: jok
your password········
welcome back


In [32]:
import os
if proceed == 1:
    
    with open('file_names.txt','r') as fn:
        files = [i.strip() for i in fn]
    
    user_files = [i for i in files if user in i]
    
    if len(user_files) > 0:
        file_exists = 1
    else:
        file_exists = 0
    
    if file_exists == 0:
        print('You do not have any files available')
        action = input('2 to create new, 4 for others')
    elif file_exists == 1:
        print(f'You have these files {user_files} available')
        action = input('0 to update, 1 to read, 2 add new file, 3 delete existing files, 4 others')
    li = [str(0),str(1),str(2),str(3),str(4)]
    while action not in li:
        print(f'no action taken as action input of {action} is invalid')
        action = input('enter 0 to update budgeting file, 1 to create new, 2 to read')
    action = int(action)
    
    if action == 0:
        update(user_files)
    elif action == 1:
        while action == 1:
            df = read(user_files)
            print(df)
            checker = input('enter 0 to read other files 1 to end')
            while checker not in [str(0),str(1)]:
                checker = input('ONLY ENTER 0 for read other files and 1 to end')
            if checker == '1':
                action = 'end'
        action = 1
    elif action == 2:
        csvFile = input('enter a file name ')
        csvFile = csvFile.replace(' ','_').replace('.','_')
        csvFile = user + '_' + csvFile + '.csv'
        create_empty_file(csvFile)
        update(csvFile)
        
        with open('file_names.txt','w') as fn:
            files.append(csvFile)
            if len(files) == 1:
                fn.write(f'{files[0]}\n')
            else:
                fn.write('\n'.join(files))
                
    elif action == 3:
        csvFile = input(f'which of the following file {user_files} would u like to delete? ')    
        while csvFile not in user_files:    
            csvFile = input(f'wrong input, select file from {user_files} to delete')
        with open('file_names.txt','w') as fn:
            files.pop(files.index(csvFile))
            if len(files) == 1:
                fn.write(f'{files[0]}\n')
            else:
                fn.write('\n'.join(files))
        os.remove(csvFile)
        print('file removed')
    elif action == 4:
        print('please email administrator for your request at thisemailisfake@fake.fakekekeke')

You have these files ['jok_jok.csv'] available
0 to update, 1 to read, 2 add new file, 3 delete existing files, 4 others4
please email administrator for your request at thisemailisfake@fake.fakekekeke


In [ ]:
# conduct datascience or exploration on the created csv files.